## Introduction

This notebook is about a classification of mushrooms into edible and poisonous categories. The data set was taken form the UCI repository, the origin of its records is The Audubon Society Field Guide to North American Mushrooms (1981). G. H. Lincoff (Pres.), New York: Alfred A. Knopf, and it was donated by Jeff Schlimmer.

This data set includes descriptions of hypothetical samples corresponding to 23 species of gilled mushrooms in the Agaricus and Lepiota Family (pp. 500-525). Each species is identified as definitely edible, definitely poisonous, or of unknown edibility and not recommended. This latter class was combined with the poisonous one. The Guide clearly states that there is no simple rule for determining the edibility of a mushroom; no rule like "leaflets three, let it be" for Poisonous Oak and Ivy. (from UCI)

The goal of this notebook is partly to refute the very Guide's statement above by showing how almost all of the classification models get a perfect score. We will see how the reason for the perfect score yields a very simple rule of how to easily classify mushrooms without any need of statistical tools. As a consequence, the primary idea of this notebook is to do some features reduction in order to seek for interesting variations that would allow us to construct and analyze models with imperfect but yet good scores. This notebook describes with comments a step-by-step process of selecting 3 variations of models.

### Attribute information (from the UCI page):
1. cap-shape: bell=b, conical=c, convex=x, flat=f, knobbed=k, sunken=s
2. cap-surface: fibrous=f, grooves=g, scaly=y, smooth=s
3. cap-color: brown=n, buff=b, cinnamon=c, gray=g, green=r, pink=p, purple=u, red=e, white=w, yellow=y
4. bruises?: bruises=t, no=f
5. odor: almond=a, anise=l, creosote=c, fishy=y, foul=f, musty=m, none=n, pungent=p, spicy=s
6. gill-attachment: attached=a, descending=d, free=f, notched=n
7. gill-spacing: close=c, crowded=w, distant=d
8. gill-size: broad=b, narrow=n
9. gill-color: black=k, brown=n, buff=b, chocolate=h, gray=g, green=r, orange=o, pink=p, purple=u, red=e, white=w, yellow=y
10. stalk-shape: enlarging=e, tapering=t
11. stalk-root: bulbous=b, club=c, cup=u, equal=e, rhizomorphs=z, rooted=r, missing=?
12. stalk-surface-above-ring: fibrous=f, scaly=y, silky=k, smooth=s
13. stalk-surface-below-ring: fibrous=f, scaly=y, silky=k, smooth=s
14. stalk-color-above-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
15. stalk-color-below-ring: brown=n, buff=b, cinnamon=c, gray=g, orange=o, pink=p, red=e, white=w, yellow=y
16. veil-type: partial=p, universal=u
17. veil-color: brown=n, orange=o, white=w, yellow=y
18. ring-number: none=n, one=o, two=t
19. ring-type: cobwebby=c, evanescent=e, flaring=f, large=l, none=n, pendant=p, sheathing=s, zone=z
20. spore-print-color: black=k, brown=n, buff=b, chocolate=h, green=r, orange=o, purple=u, white=w, yellow=y
21. population: abundant=a, clustered=c, numerous=n, scattered=s, several=v, solitary=y
22. habitat: grasses=g, leaves=l, meadows=m, paths=p, urban=u, waste=w, woods=d

## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.preprocessing import OneHotEncoder

from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import VarianceThreshold, SelectPercentile, f_classif, chi2, mutual_info_classif

from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

from sklearn.metrics import make_scorer, fbeta_score, confusion_matrix, classification_report, roc_curve, auc

from lime import lime_tabular
import random

## Loading data set

In [ ]:
df = pd.read_csv("../input/mushroom-classification/mushrooms.csv")

## Exploratory analysis

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Lets see the unique values in the datafrmame
print("Unique values:")
for col in df.columns:
    print(col + ":", df[col].unique())

#### Missing values (compared to the UCI attribute information):

- gill-attachment: d, n.
- gill-spacing: d.
- stalk-root: u, z.
- veil-type: u.
- ring-type: c, s, z.

In [ ]:
df.info()

In [ ]:
# Checking for missing values
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
# Checking for duplicate samples
df[df.duplicated()].shape[0]

In [ ]:
# Count-plots all of the features in the dataframe
def features_countplot(df):
    fig, axs = plt.subplots(8, 3, figsize=(14,30), constrained_layout=True)
    for i, f in enumerate(df.columns):
        sb.countplot(x=f, data=df, ax=axs[i//3][i%3])
    plt.show()

In [ ]:
features_countplot(df)

The class distribution is balanced; and since all of the features are categoric there's no need to scale the data.

#### Inspecting rare values:

In [ ]:
# Prints a frequency of values under a threshold
def values_freq(df, n):
    print("Number of values with frequency of less than", str(n) + ":")
    for col in df.columns:
        for val in list(df[col].unique()):
            if df[col][df[col] == val].count() < n:
                print(col, "of type", val + ":", df[col][df[col] == val].count())

In [ ]:
values_freq(df, 10)

# Preprocessing

In [ ]:
# As we've noticed above, there's only 1 veil-type so we can drop the whole feature out
df.drop("veil-type", axis=1, inplace=True)

### Data encoding:

In [ ]:
# Gets a OneHotEncoder matrix and returns it as a dataframe with the dummy columns of df
def ohe_to_df(ohe, df):
    columns = pd.get_dummies(df, drop_first=True).columns
    ohe_df = pd.DataFrame(ohe, columns=columns[1:]).astype("int")
    ohe_df.insert(0, "class", df["class"])
    ohe_df["class"].replace({"e":1, "p":0}, inplace=True)
    return ohe_df

In [ ]:
df2_encoder = OneHotEncoder(sparse=False, drop="first").fit(df.drop("class", axis=1))
df2 = ohe_to_df(df2_encoder.transform(df.drop("class", axis=1)), df)
df2.head()

## Analysis

In [ ]:
plt.figure(figsize=(22,22))
sb.heatmap(df2.corr(), cmap='coolwarm')

#### Pairs of highly correlated features:

In [ ]:
# Prints pairs of features with a correlation that is greater than or equal to the given coefficient (in absolute value)
def df_corr_coeff(df, coeff):
    table = []
    upper = df.corr().where(np.triu(np.ones(df.corr().shape), k=1).astype(np.bool))
    for col in df.corr().columns:
        for i, val in enumerate(list(upper[col].dropna().values)):
            if abs(val) >= coeff:
                table.append((upper[col].dropna().name, df.corr().columns[i], val))
    table = pd.DataFrame(table, columns=["Feature 1", "Feature 2", "Correlation"])
    return table

In [ ]:
df_corr_coeff(df2, 0.9)

In [ ]:
# Deletes a feature for every pair of perfectly correlated features (disregarding the target feature)
def del_perfect_corr(df):
    
    features = df_corr_coeff(df, 1)
    if features.shape[0] == 0:
        return df
    features_groups = [{features["Feature 1"][0], features["Feature 2"][0]}]
    for i in range(1, features.shape[0]):
        if features["Feature 1"][i] not in set.union(*features_groups) and features["Feature 2"][i] not in set.union(*features_groups):
            features_groups.append({features["Feature 1"][i], features["Feature 2"][i]})
        elif features["Feature 1"][i] in set.union(*features_groups) and features["Feature 2"][i] in set.union(*features_groups):
            continue
        else:
            for group in features_groups:
                if features["Feature 1"][i] in group or features["Feature 2"][i] in group:
                    group.add(features["Feature 1"][i])
                    group.add(features["Feature 2"][i])
                    break
    for group in features_groups:
        group.pop()
        df = df.drop(group, axis=1)        
    return df

In [ ]:
# In every group of perfectly correlated features we want to drop from the dataframe all features but one
df2 = del_perfect_corr(df2)

## Take 0: Initial evaluation

### Train-test split

In [ ]:
X = df2.drop("class", axis=1)
y = df2["class"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

# Creating models

In [ ]:
# Runs classification models using GridSearchCV or RandomizedSearhCV and outputs their best estimators and a table with their best scores and parameters
def run_models(X, y, searchCV):
    
    grid_params = {"Logistic Regression": {"model": LogisticRegression(), "params": {"C": [.1, 1, 8, 9, 10], "max_iter": [300], "solver": ["lbfgs", "liblinear"]}},
                     "SGD": {"model": SGDClassifier(), "params": {"loss": ["modified_huber"]}},
                     "KNN": {"model": KNeighborsClassifier(), "params": {"n_neighbors": range(1,10), "weights": ["uniform", "distance"]}},
                     "SVM": {"model": SVC(), "params": {"C": [.01, .1, .5, 1, 5, 10], "kernel": ["linear", "poly", "rbf", "sigmoid"]}},
                     "Naive Bayes": {"model": GaussianNB(), "params": {}},
                     "Decision Tree": {"model": DecisionTreeClassifier(), "params": {"criterion": ["gini", "entropy"]}},
                     "Random Forest": {"model": RandomForestClassifier(), "params": {"n_estimators": [50, 100, 200], "criterion": ["gini", "entropy"]}}}
    scores = []
    estimators = {}
    for model, mp in grid_params.items():
        clf = searchCV(mp["model"], mp["params"], scoring=make_scorer(fbeta_score, beta=.5), n_jobs=-1, cv=5)
        clf.fit(X, y)
        scores.append({"Model": model, "Best Score": clf.best_score_, "Best Parameters": clf.best_params_})
        estimators[model] = clf.best_estimator_
    print(pd.DataFrame(scores, columns=["Model", "Best Score", "Best Parameters"]).to_string())
    return estimators

We take the f_beta function with beta = 0.5 as the scoring function to lend more weight to precision because in this dataset it's important to keep the FP low (poisonous mushrooms that were falsely predicted to be edible).

In [ ]:
estimators = run_models(X_train, y_train, GridSearchCV)

In [ ]:
# Gets a dictionary of estimators and returns a corresponding dictionary of predictions over a test set
def pred_dict(estimators, X_test):
    return dict(zip(estimators.keys(), [x.predict(X_test) for x in estimators.values()]))

In [ ]:
predictions = pred_dict(estimators, X_test)

### Confusion matrices and Classification reports

In [ ]:
# Plots confusion matrices for a dictionary of estimators
def confusion_mat(y_test, predictions):
    plt.figure(figsize=(18,12))
    plt.suptitle("Confusion Matrices",fontsize=30)
    plt.subplots_adjust(wspace = 0.4, hspace= 0.4)
    
    for i, model in enumerate(predictions.keys()):
        plt.subplot(3,3,i + 1)
        plt.title(model, fontsize=15, pad=10)
        sb.heatmap(confusion_matrix(y_test, predictions[model]), annot=True, cmap="Blues", 
                   fmt="d",cbar=False, annot_kws={"size": 25})
    plt.show()

In [ ]:
# Prints classification reports for a dictionary of estimators
def classification_rep(y_test, predictions):
    print("Classification Reports".center(125, "="))
    for i, model in enumerate(predictions.keys()):
        print(model.center(55, " "))
        print(classification_report(y_test, predictions[model]))
        print("*" * 54)

We have no need to use these functions for now because we got a perfect score, so we should further analyse the data to see what causes it.

# Feature Selection and Distribution

Let's see how the features from the original dataframe are distributed in the target.

In [ ]:
# Plots the distribution of the features in the target
def feature_target_dist(df):
    fig, axs = plt.subplots(7, 3, figsize=(14,30), constrained_layout=True)
    for i, f in enumerate(df.drop("class", axis=1).columns):
        sb.countplot(x=f, hue="class", data=df, ax=axs[i//3][i%3])
    plt.show()

In [ ]:
feature_target_dist(df)

We can see several features that are distributed in the target unequally, maybe the most prominent is odor.

In [ ]:
# Prints the frequency of a given feature of a dataframe when it's grouped by the target
def feature_freq(df, feature):
    df = pd.get_dummies(df[["class", feature]], columns=[feature])
    for f in df.drop("class", axis=1).columns:
        fd = pd.DataFrame(df[["class", f]][df[f] == 1].groupby("class").count())
        fd["percent"] = fd[f] / fd[f].sum() * 100
        print(fd)
        print("*************************")

In [ ]:
feature_freq(df, "odor")

We see that all of the mushrooms with almond or anise odor are edible; all of the mushrooms with creosote, fishy, foul, musty, pungent and spicy odors are poisonous; and among the mushrooms without an odor almost 97% are edible. So the distribution of the  "odor" feature is strongly compatible with the distribution of the target, and apparently it's one of the causes that our models have perfect score. This observation actually draws a simple rule to determine the edibility of a mushroom, as opposed to the aforementioned Guide's statement. Let's run some features importance tests to see it more clearly.

In [ ]:
# Plots statistical tests between the feature matrix X and the target vector y for n features with the highest scores
def stat_scores(X, y, n):
    
    fc = f_classif(X, y)
    chi = chi2(X, y)
    mi = mutual_info_classif(X, y, random_state=10)
    rfi = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=10).fit(X, y).feature_importances_
    eti = ExtraTreesClassifier(n_estimators=100, random_state=10).fit(X, y).feature_importances_
    
    plt.title("ANOVA F-value stats")
    plot_feature_rank(X, fc[0], n)
    
    plt.title("Chi-squared stats")
    plot_feature_rank(X, chi[0], n)
    
    plt.title("Mutual Information stats")
    plot_feature_rank(X, mi, n)
    
    plt.title("Random Forest Importance")
    plot_feature_rank(X, rfi, n)
    
    plt.title("Extra Trees Importance")
    plot_feature_rank(X, eti, n)

In [ ]:
# Plots a chart of n important features from X using stat test
def plot_feature_rank(X, stat, n):
    pd.Series(stat).sort_values(ascending=False).nlargest(n).plot.bar(figsize = (16,4))
    plt.xticks(range(0,n), X.columns[pd.Series(stat).sort_values(ascending=False).nlargest(n).index], rotation=20)
    plt.show()

In [ ]:
stat_scores(X_train, y_train, 10)

We can see the 'none odor' has significant dominance compared to the rest of the values, and hence the next step would be to drop the whole feature out and see where it leads us. Before that we just add a small section of LIME interpretability that we will use later on.

### Interpretability:

In [ ]:
# Explaining the model with LIME by locally choosing a random test sample (DOES NOT WORK FOR SVM)
def lime_exp(X_train, X_test, model, n=10):
    explainer = lime_tabular.LimeTabularExplainer(X_train.values, 
                                                  feature_names=X_train.columns)
    exp = explainer.explain_instance(X_test.iloc[random.randint(0, len(X_test) - 1)], 
                                     model.predict_proba, num_features=n)
    exp.show_in_notebook()

In [ ]:
lime_exp(X_train, X_test, estimators["KNN"])

## Take 1 (anosmic): Data without the odor feature

In [ ]:
original_df3 = df.drop("odor", axis=1)

df3_encoder = OneHotEncoder(sparse=False, drop="first").fit(original_df3.drop("class", axis=1))
df3 = ohe_to_df(df3_encoder.transform(original_df3.drop("class", axis=1)), original_df3)

df3 = del_perfect_corr(df3)

X2 = df3.drop("class", axis=1)
y2 = df3["class"]
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=10)

estimators2 = run_models(X_train2, y_train2, GridSearchCV)

We still get a perfect score in most of the models, and that's probably because of other features with high importance. Now, instead of shaving a few more features with high importance or low variance off of the dataframe, we should make it interesting and establish thinner variations of the data set, from which we shall focus on those that don't yield a prefect score but yet a decent one.

## Take 2 (anti-anosmic): Data with only the odor feature

In [ ]:
original_df4 = df[["class", "odor"]]

df4_encoder = OneHotEncoder(sparse=False, drop="first").fit(original_df4.drop("class", axis=1))
df4 = ohe_to_df(df4_encoder.transform(original_df4.drop("class", axis=1)), original_df4)

df4 = del_perfect_corr(df4)

In [ ]:
plt.figure(figsize=(15,15))
sb.heatmap(df4.corr(), cmap='coolwarm', annot=True, vmin=-1, annot_kws={"fontsize":15})

In [ ]:
X3 = df4.drop("class", axis=1)
y3 = df4["class"]
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.3, random_state=10)

estimators3 = run_models(X_train3, y_train3, GridSearchCV)

Considering what we've seen above it's not very surprising that we get almost a perfect score (that is also identical across all of the models), and therefore we have no use for this variation.

## Take 3: Colors only

In [ ]:
original_df5 = df[["class"] + [i for i in df.columns if "color" in i]]

df5_encoder = OneHotEncoder(sparse=False, drop="first").fit(original_df5.drop("class", axis=1))
df5 = ohe_to_df(df5_encoder.transform(original_df5.drop("class", axis=1)), original_df5)

df5 = del_perfect_corr(df5)

In [ ]:
plt.figure(figsize=(22,22))
sb.heatmap(df5.corr(), cmap='coolwarm')

In [ ]:
X4 = df5.drop("class", axis=1)
y4 = df5["class"]
X_train4, X_test4, y_train4, y_test4 = train_test_split(X4, y4, test_size=0.3, random_state=10)

estimators4 = run_models(X_train4, y_train4, GridSearchCV)

Well, that's an interesting variation. The KNN model got the highest score, so as for now that's the model to go with.

### Attempts to improve the models:

We will use several ways to try improving the models.

In [ ]:
# Runs a variance threshold algorithm and returns the features with a variance that is lower than the threshold
def var_threshold(X, threshold):
    vt = VarianceThreshold(threshold)
    vt.fit(X)
    low_var_features = [x for x in X.columns if x not in X.columns[vt.get_support()]]
    print("Feaures with variance of less than " + str(threshold) + ":", ", ".join(low_var_features))
    print("Number of remaining features:", sum(vt.get_support()))
    return low_var_features

In [ ]:
# Runs the KNN model and plots a chart of K values from 1 to n with their scores
def best_knn(X, y, n):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
    scoreList = []
    for i in range(1,n + 1):
        knn = GridSearchCV(KNeighborsClassifier(n_neighbors = i, n_jobs=-1), {"weights": ["uniform", "distance"]}, scoring=make_scorer(fbeta_score, beta=.5), cv=5)
        knn.fit(X_train, y_train)
        scoreList.append(knn.best_score_)
    plt.plot(range(1,n + 1), scoreList)
    plt.xticks(np.arange(1,n + 1,1))
    plt.xlabel("K value")
    plt.ylabel("Score")
    plt.show()
    print("Maximum KNN Score:", max(scoreList), "for K =", scoreList.index(max(scoreList)) + 1)

In [ ]:
# Fixes the multicolinearity of a dataframe by dropping features with high Variance Inflation Factor (VIF) until all features have a VIF of less than n. Returns the updated dataframe and the VIF dataframe with VIF values of the remaining features
def fix_multicolinearity(df, n):
    
    vif = pd.DataFrame()
    vif["features"] = df.columns
    vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    while any(i > n for i in vif["VIF"].values):
        for f in range(1, df.shape[1]):
            if vif.iloc[f][1] > n:
                df = df.drop(vif.iloc[f][0], axis=1)
                break
        vif = pd.DataFrame()
        vif["features"] = df.columns
        vif["VIF"] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    return df, vif

In [ ]:
X4_1 = X4.drop(var_threshold(X4, 0.005), axis=1)
X_train4_1, X_test4_1, y_train4_1, y_test4_1 = train_test_split(X4_1, y4, test_size=0.3, random_state=10)
estimators4_1 = run_models(X_train4_1, y_train4_1, GridSearchCV)

We can see that by removing some features with low variance still none of the models is better than our previous KNN, and therefore removing low variance features is not a good idea here (tested with a variety of thresholds).

In [ ]:
X4_2 = SelectPercentile(chi2, percentile=90).fit_transform(X4, y4)
X_train4_2, X_test4_2, y_train4_2, y_test4_2 = train_test_split(X4_2, y4, test_size=0.3, random_state=10)
estimators4_2 = run_models(X_train4_2, y_train4_2, GridSearchCV)

Selecting 90% of the best features doesn't improve our models either (tested with a variety of functions and percentiles).

In [ ]:
best_knn(X4, y4, 30)

Increasing the n neighbors for the KNN doesn't improve it.

In [ ]:
new_df5, df5_vif = fix_multicolinearity(df5, 10)
X4_3 = new_df5.drop("class", axis=1)
y4_3 = new_df5["class"]
X_train4_3, X_test4_3, y_train4_3, y_test4_3 = train_test_split(X4_3, y4_3, test_size=0.3, random_state=10)

estimators4_3 = run_models(X_train4_3, y_train4_3, GridSearchCV)

Fixing the multicolinearity using VIF does improve the KNN by a bit.

In [ ]:
predictions4 = pred_dict(estimators4, X_test4)

In [ ]:
confusion_mat(y_test4, predictions4)

We get an interesting observation from the confusion matrices. FP in these models means that a poisonous mushroom was predicted to be edible, so ideally we would want to minimize the FP as much as possible. KNN got the best score overall but its FP is not the lowest. Naive Bayes got the worst score with a huge amount of FN but it has no FP at all, so it's basically the safest model!

In [ ]:
predictions4_3 = pred_dict(estimators4_3, X_test4_3)

In [ ]:
confusion_mat(y_test4_3, predictions4_3)

The confusion matrices of both the fixed multicolinearity KNN and the original KNN are the same, so we'll continue with the previous KNN for now.

# Postprocessing

### Threshold optimization:

In this section we would want to take our current estimators and optimize their decision thresholds in order to decrease the FP rate. We would do that in 3 steps:

- Creating predict-proba sets by splitting them over the train set using KFold (to avoid over-fitting).
- Using these sets to calculate f_0.5 scores on all of the possible thresholds of the corresponding untrained estimators (with the same parameters).
- For each estimator, taking the threshold that yields the highest score.

In [ ]:
# Trains a model using KFold cross-validation on X and y and returns a dataframe with the labels and concatinated predict_probas for each split. The idea is to use it to find an optimal threshold without an overfitting risk on the test set
def kfold_predict_proba(X, y, model):
    kf = KFold(n_splits=5, shuffle=True, random_state=10)
    clf = model
    pp_df = pd.DataFrame(columns=["Label", "Predict_proba"])
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        clf.fit(X_train, y_train)
        pp_df = pd.concat([pp_df, pd.DataFrame({"Label": y_test, "Predict_proba": clf.predict_proba(X_test)[:,1]})], axis=0)
    return pp_df

In [ ]:
# Plots ROC curves of estimators using K-fold predcit_proba
def roc_curves(X, y, estimators):
    
    estimators2 = dict(estimators)
    estimators2.pop("SVM")
    fig, axs = plt.subplots(2, 3, figsize=(15,9), constrained_layout=True)
    for i, clf in enumerate(estimators2.values()):
        clf_pp = kfold_predict_proba(X, y, clf)
        label = clf_pp["Label"].tolist()
        pp = clf_pp["Predict_proba"].tolist()
        fpr, tpr, thresholds = roc_curve(label, pp)
        area = auc(fpr, tpr)
        axs[i//3][i%3].plot(fpr, tpr, linestyle="-", label='ROC curve (area = %0.2f)' % area)
        axs[i//3][i%3].set(xlabel="False Positive Rate", ylabel="True Positive Rate", title=list(estimators2.keys())[i])
        axs[i//3][i%3].legend()
    fig.suptitle("ROC Curves", fontsize=20)
    plt.show()

In [ ]:
# Gets a dictionary of trained estimators and returns a dictionary of the same estimators but untrained and with the same parameters
def untrain_models(estimators):
    new_clfs = []
    for clf in estimators.values():
        new_clfs.append(eval(type(clf).__name__)().set_params(**clf.get_params()))
    return dict(zip(estimators.keys(), new_clfs))

In [ ]:
# Given lists of predict_proba and labels, returns the best threshold under f_0.5 scoring
def best_threshold(pred_df):
    label = pred_df["Label"].tolist()
    pred_proba = pred_df["Predict_proba"].tolist()
    fpr, tpr, thresholds = roc_curve(label, pred_proba)
    scores = np.array([fbeta_score(label, pred_proba >= thr, beta=.5) for thr in thresholds])
    return thresholds[scores.argmax()]

In [ ]:
# Gets a dictionary of optimal thresholds for given estimators and X, y train sets
def best_thresholds_dict(X, y, estimators):
    thresholds = []
    for clf in untrain_models(estimators).values():
        if type(clf).__name__ == "SVC":
            thresholds.append(0.5)
        else:
            thresholds.append(best_threshold(kfold_predict_proba(X, y, clf)))
    return dict(zip(estimators.keys(), thresholds))

In [ ]:
# Model prediction by a given decision threshold
def predict_by_threshold(model, threshold, test):
    if type(model).__name__ == "SVC":
        return model.predict(test)
    return (model.predict_proba(test)[:,1] >= threshold).astype("int")

In [ ]:
# Returns a dictionary of models predictions by the corresponding decision thresholds
def pred_by_threshold_dict(estimators, thresholds, test):
    preds = []
    for i, clf in enumerate(estimators.values()):
        preds.append(predict_by_threshold(clf, thresholds[list(thresholds.keys())[i]], test))
    return dict(zip(estimators.keys(), preds))

In [ ]:
thresholds4 = best_thresholds_dict(X_train4, y_train4, estimators4)

In [ ]:
predictions4 = pred_by_threshold_dict(estimators4, thresholds4, X_test4)

In [ ]:
confusion_mat(y_test4, predictions4)

In [ ]:
classification_rep(y_test4, predictions4)

We optimized our thresholds by the f_0.5 scoring function to increase the precision in order to decrease the FP, and we got the following results on the test set:
- The FP of the Logistic Regression, SGD, KNN, Decision Tree and Random Forest were decreased.
- The FP of the Naive Bayes was increased (apparently because it previously had 100% precision and now we forced some FP).
- The confusion matrix of the SVM remained the same (because SVM is not a probabilistic algorithm and hence it has no decision threshold).

We can draw the following conclusions:
- If we were to seek for the safest model, the previous Naive Bayes has no FP at all, but on the other hand it does have 871 FN, which eventually granted it a very low score. A good candidate for a very safe model would be the current Logistic Regression which has only a few more FP than the previous Naive Bayes but more than 500 FN less.
- Even though the KNN got the highest score, the Decision Tree and Random Forest were previously very close to it with only a 0.002 difference. Now we can see their current "thresholded" versions have 12 more FN but 14 less FP, which to me grants them to be the final model to end up with.

In [ ]:
colors_clf = estimators4["Random Forest"]
colors_threshold = thresholds4["Random Forest"]

In [ ]:
# Returns the indices of the removed columns during the preprocessing stage
def removed_indices(original_df, df):
    cols = pd.get_dummies(original_df.drop("class", axis=1), drop_first=True).columns
    return [cols.tolist().index(x) for x in cols if x not in df.columns]

In [ ]:
# Gets a dataframe and its deleted indices during the preprocessing and returns the updated unique values as a matrix
def unique_values2(df, del_idx):
    
    unique_mat = []
    del_col = pd.get_dummies(df.drop("class", axis=1), drop_first=True).columns[del_idx]
    for f in df.drop("class", axis=1).columns:
        values = df[f].unique().tolist()
        for c in del_col:
            if f == c[:-2]:
                values.remove(c[-1])
        unique_mat.append(values)
    return unique_mat

To wrap everything up, I made a class that takes the final model and uses it to make predictions.

In [ ]:
# Class for the final model we achieved after all the analysis. Gets the estimator, its best decision threshold, the original dataframe, the OHE fitted object of the original dataframe and the indices of the removed columns during the preprocessing
class MushroomModel:
    
    def __init__(self, estimator, threshold, df, encoder, del_idx):
        self.estimator = estimator
        self.threshold = threshold
        self.df = df
        self.encoder = encoder
        self.del_idx = del_idx
        self.uniques = unique_values2(df, del_idx)
        
    def random_predict(self):
        sample = []
        for i, col in enumerate(self.df.drop("class", axis=1).columns):
            sample.append(random.choice(self.uniques[i]))
            print(col + ":", sample[-1])
        return self.predict(sample)
        
    def sample_fit(self, sample):
        sample = self.encoder.transform(np.array(sample).reshape(1,-1))[0]
        sum_check = sample.sum()
        sample = np.array([x for i, x in enumerate(sample) if i not in self.del_idx]).reshape(1,-1)
        if sum_check != sample.sum():
            raise ValueError("Invalid input")
        return sample
    
    def predict(self, sample):
        sample = self.sample_fit(sample)
        est_name = type(self.estimator).__name__
        if  est_name == "SVC" or est_name == "KMeans":
            return self.estimator.predict(sample)
        return (self.estimator.predict_proba(sample)[:,1] >= self.threshold).astype("int")

In [ ]:
colors_model = MushroomModel(colors_clf, colors_threshold, original_df5, df5_encoder, removed_indices(original_df5, df5))

In [ ]:
sample = ['n','k','n','n','w','k']

In [ ]:
colors_model.predict(sample)

In [ ]:
lime_exp(X_train4, pd.DataFrame(colors_model.sample_fit(sample)), colors_clf)

In [ ]:
colors_model.random_predict()

In [ ]:
lime_exp(X_train4, X_test4, colors_clf)

## Take 4: Area (population + habitat)

In [ ]:
original_df6 = df[["class", "population", "habitat"]]

df6_encoder = OneHotEncoder(sparse=False, drop="first").fit(original_df6.drop("class", axis=1))
df6 = ohe_to_df(df6_encoder.transform(original_df6.drop("class", axis=1)), original_df6)

df6 = del_perfect_corr(df6)

In [ ]:
plt.figure(figsize=(15,15))
sb.heatmap(df6.corr(), cmap='coolwarm', annot=True, vmin=-1, annot_kws={"fontsize":15})

In [ ]:
X5 = df6.drop("class", axis=1)
y5 = df6["class"]
X_train5, X_test5, y_train5, y_test5 = train_test_split(X5, y5, test_size=0.3, random_state=10)

estimators5 = run_models(X_train5, y_train5, GridSearchCV)

Another interseting variation that is very thin in terms of features and yet yields not very bad scores. We shall go through the whole process like we did in the last variation, and meanwhile the candidate for the best model is the SVM.

### Attempts to improve the models:

In [ ]:
X5_1 = X5.drop(var_threshold(X5, 0.05), axis=1)
X_train5_1, X_test5_1, y_train5_1, y_test5_1 = train_test_split(X5_1, y5, test_size=0.3, random_state=10)
estimators5_1 = run_models(X_train5_1, y_train5_1, GridSearchCV)

We can see that removing the 5 features with a variance threshold of less than 0.05 decreases a bit the scores of the Logistic Regression and SGD but improves the KNN, SVM, Decision Tree and Random Forest models to the same score, so as for now each one of them is a candidate to be the model to go with (tested with a variety of thresholds and picked the one that yielded the best score). I think the fact we removed almost half of the features and yet received better models might imply the data set is not very balanced...

In [ ]:
X5_2 = SelectPercentile(chi2, percentile=90).fit_transform(X5_1, y5)
X_train5_2, X_test5_2, y_train5_2, y_test5_2 = train_test_split(X5_2, y5, test_size=0.3, random_state=10)
estimators5_2 = run_models(X_train5_2, y_train5_2, GridSearchCV)

Selecting 90% of the best features doesn't improve our models (tested with a variety of functions and percentiles).

In [ ]:
best_knn(X5_1, y5, 30)

Increasing the n neighbors for the KNN doesn't yield a better KNN.

In [ ]:
new_df6, df6_vif = fix_multicolinearity(df6, 10)
X5_3 = new_df6.drop("class", axis=1).drop(var_threshold(X5, 0.05), axis=1)
y5_3 = new_df6["class"]
X_train5_3, X_test5_3, y_train5_3, y_test5_3 = train_test_split(X5_3, y5_3, test_size=0.3, random_state=10)

estimators5_3 = run_models(X_train5_3, y_train5_3, GridSearchCV)

Fixing the multicolinearity using VIF doesn't improve the models.

In [ ]:
predictions5 = pred_dict(estimators5, X_test5)

In [ ]:
predictions5_1 = pred_dict(estimators5_1, X_test5_1)

In [ ]:
confusion_mat(y_test5, predictions5)

In [ ]:
confusion_mat(y_test5_1, predictions5_1)

Putting a variance threshold like we did increased the FP of the Logistic Regression, SGD, KNN and Naive Bayes but decreased it in the SVM, Decision Tree and Random Forest. On the other hand, the FN of the last ones have been significantly increased. Like before, we can take the previous Naive Bayes as the safest model with only 21 FP. But if we want to choose the model with the least FP and yet with a decent amount of FN we should choose the previous KNN. Lets see how the confusion matrix on the train test looks like.

In [ ]:
confusion_mat(y_train5, pred_dict(estimators5, X_train5))

We can see the KNN keeps its high precision on the train set with almost an opposite proportion of FP/FN compared to the rest of the good models. I also tested it with the variance threshold but it yielded worse results, so meanwhile the original KNN is the best model.

### Threshold optimization:

In [ ]:
roc_curves(X_train5, y_train5, estimators5)

In [ ]:
thresholds5 = best_thresholds_dict(X_train5, y_train5, estimators5)

In [ ]:
predictions5 = pred_by_threshold_dict(estimators5, thresholds5, X_test5)

In [ ]:
confusion_mat(y_test5, predictions5)

In [ ]:
classification_rep(y_test5, predictions5)

Optimizing the decision threshold didn't yield a better model than the KNN but made the Random Forest and Decision Tree match its score, and hence the KNN is the model we end up with.

In [ ]:
area_clf = estimators5["KNN"]
area_threshold = thresholds5["KNN"]

In [ ]:
area_model = MushroomModel(area_clf, area_threshold, original_df6, df6_encoder, removed_indices(original_df6, df6))

In [ ]:
area_model.random_predict()

In [ ]:
sample2 = ['c', 'l']

In [ ]:
lime_exp(X_train5, pd.DataFrame(area_model.sample_fit(sample2)), area_clf)

In [ ]:
lime_exp(X_train5, X_test5, area_clf)

Just for the sake of experimenting I also created XGBoost model. 

In [ ]:
def xgboost_model(X, y, searchCV):
    
    params = {
        'min_child_weight': [0.1, 0.5, 1, 5],
        'gamma': [0.01, 0.1, 0.5, 1, 1.5, 2],
        'subsample': [0.7, 0.8, 0.9],
        'colsample_bytree': [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 2],
        'max_depth': [2, 3, 4, 10]
        }
    xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic', silent=True, nthread=1)
    clf = searchCV(xgb, params, scoring=make_scorer(fbeta_score, beta=.5), n_jobs=-1, cv=5)
    clf.fit(X, y)
    print("XGBoost best score:", clf.best_score_)
    print("XGBoost best parameters:", clf.best_params_)
    return clf.best_estimator_

In [ ]:
xgb = xgboost_model(X_train5, y_train5, RandomizedSearchCV)

In [ ]:
confusion_matrix(y_test5, xgb.predict(X_test5))

We get the same confusion matrix as the KNN.

## Take 5: Unsupervised (clustering)

Now we are back to the original dataframe that we encoded with a OneHotEncoder (df2) but leaving the target feature out this time, and we shall see if dividing the data into 2 clusters using KMeans and Gaussian Mixture (separately) yields a good approximation to the original target.

In [ ]:
# Calculates an f_0.5 score between the original labels feature and a given clustering labels vector
def clustering_score(labels, cl_labels):
    cm = confusion_matrix(labels, cl_labels)
    tp = cm[1][1]
    fp = cm[0][1]
    fn = cm[1][0]
    return (1.25 * tp) / (1.25 * tp + 0.25 * fn + fp)

In [ ]:
# Creates clustering models for the dataframe by using 2 clusters that would substitute the original binary target and outputs the clustering labels vectors and a table with the models scores
def clustering_model(df):
    
    kmeans = KMeans(n_clusters=2, random_state=0).fit(df.drop("class", axis=1))
    gmm = GaussianMixture(n_components=2, random_state=0).fit(df.drop("class", axis=1))
    kmeans_pred = kmeans.predict(df.drop("class", axis=1))
    gmm_pred = gmm.predict(df.drop("class", axis=1))
    kmeans_score = clustering_score(df["class"], kmeans_pred)
    gmm_score = clustering_score(df["class"], gmm_pred)
    estimators = {"KMeans": kmeans, "Gaussian Mixture": gmm}
    print(pd.DataFrame([["KMeans", kmeans_score],["Gaussian Mixture", gmm_score]], columns=["Model", "Score"]))
    return estimators

The models are initialized with a specific random_state to make sure the clustering labels correspond to the original labels (1 to 1 and 0 to 0), and also because it gives them the best score.

In [ ]:
estimators6 = clustering_model(df2)

In [ ]:
predictions6 = pred_dict(estimators6, df2.drop("class", axis=1))

In [ ]:
confusion_mat(df2["class"], predictions6)

In [ ]:
classification_rep(df2["class"], predictions6)

The models separate the data almost the same, and unfortunately they both have a high FP rate. The score of the KMeans is a little better so that's the model we end up with.

In [ ]:
clusters_model = MushroomModel(estimators6["KMeans"], 0.5, df, df2_encoder, removed_indices(df, df2))

In [ ]:
clusters_model.random_predict()

### Ending remark

One could mess with the beta value of the f_beta function a little more to try to get better precision results.